In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

#for statement check            
#df.head
#print(max(df['id_path']))

(16127, 31)


In [3]:
!pip install linearmodels  
from linearmodels import PanelOLS
import statsmodels.api as sm

# Col1: pr & ctrl_exp

In [4]:
#table 3 = pr
#(df['turin_pr_sample']==1)&(df['ctrl_exp_turin_pr_sample']==1)
#outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

#winning discount
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 
#value chekced

#completion time delay
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.delay_ratio, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#cost_overrun
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.overrun_ratio, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#days to award
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.days_to_award, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#all value checked
#tbc: comparision with stata

C:\Users\thxkn\anaconda3\lib\site-packages\linearmodels\panel\model.py:1669: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

year.2001.0, year.2002.0, year.2003.0, year.2004.0, year.2005.0, year.2006.0

  AbsorbingEffectWarning,
C:\Users\thxkn\anaconda3\lib\site-packages\linearmodels\utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:               discount   R-squared:                        0.3856
Estimator:                   PanelOLS   R-squared (Between):              0.9863
No. Observations:                1355   R-squared (Within):               0.0796
Date:                Thu, Jul 02 2020   R-squared (Overall):              0.3688
Time:                        01:20:44   Log-likelihood                   -4169.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      46.371
Entities:                          18   P-value                           0.0000
Avg Obs:                       75.278   Distribution:                 F(18,1330)
Min Obs:                       27.000                                           
Max Obs:                       147.00   F-statistic (robust):             46.371
                            

# Col 3: pr & ctrl_pop

In [6]:
#winning discount
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#completion time delay
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.delay_ratio, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#cost_overrun
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.overrun_ratio, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#days to award
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.days_to_award, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#all value checekd
#tbc: comparision with stata

                          PanelOLS Estimation Summary                           
Dep. Variable:               discount   R-squared:                        0.4977
Estimator:                   PanelOLS   R-squared (Between):              0.9758
No. Observations:                 653   R-squared (Within):               0.1539
Date:                Thu, Jul 02 2020   R-squared (Overall):              0.4946
Time:                        01:22:05   Log-likelihood                   -1999.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      57.209
Entities:                          11   P-value                           0.0000
Avg Obs:                       59.364   Distribution:                  F(11,635)
Min Obs:                       16.000                                           
Max Obs:                       147.00   F-statistic (robust):             57.209
                            

# col 5: pr & ctrl_pop_exp

In [7]:
#winning discount
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#completion time delay
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.delay_ratio, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#cost_overrun
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.overrun_ratio, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#days to award
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.days_to_award, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 

#all value checked
#tbc: comparision with stata

                          PanelOLS Estimation Summary                           
Dep. Variable:               discount   R-squared:                        0.5113
Estimator:                   PanelOLS   R-squared (Between):              0.9796
No. Observations:                 567   R-squared (Within):               0.1649
Date:                Thu, Jul 02 2020   R-squared (Overall):              0.4989
Time:                        01:23:27   Log-likelihood                   -1751.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      72.184
Entities:                           8   P-value                           0.0000
Avg Obs:                       70.875   Distribution:                   F(8,552)
Min Obs:                       42.000                                           
Max Obs:                       147.00   F-statistic (robust):             72.184
                            